In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df_clicks = pd.read_csv('yoochoose-clicks.dat', header=None, names=['SessionID', 'Timestamp', 'ItemID', 'Category'])

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_clicks.head()

,SessionID,Timestamp,ItemID,Category
0,1,2014-04-07T10:51:09.277Z,214536502,0
1,1,2014-04-07T10:54:09.868Z,214536500,0
2,1,2014-04-07T10:54:46.998Z,214536506,0
3,1,2014-04-07T10:57:00.306Z,214577561,0
4,2,2014-04-07T13:56:37.614Z,214662742,0


In [4]:
len(df_clicks)

33003944

In [5]:
df_buys = pd.read_csv('yoochoose-buys.dat', header=None, names=['SessionID', 'Timestamp', 'ItemID', 'Price', 'Quantity'])

In [6]:
df_buys.head()

,SessionID,Timestamp,ItemID,Price,Quantity
0,420374,2014-04-06T18:44:58.314Z,214537888,12462,1
1,420374,2014-04-06T18:44:58.325Z,214537850,10471,1
2,281626,2014-04-06T09:40:13.032Z,214535653,1883,1
3,420368,2014-04-04T06:13:28.848Z,214530572,6073,1
4,420368,2014-04-04T06:13:28.858Z,214835025,2617,1


In [7]:
len(df_buys)

1150753

In [8]:
# what percentage of sessions result in buys
len(df_buys.groupby('SessionID').nunique()) / len(df_clicks.groupby('SessionID').nunique())

0.05510388466516154

In [9]:
# how long are sessions that result in buys vs. non-buys
# df was created and stored in .csv format containing session info

# column was created for sessionIDs in click df that also exist in buy df
# df_clicks['InBuys'] = df_clicks.SessionID.isin(df_buys.SessionID) * 1

# session info df
# df_session_info = df_clicks.groupby('SessionID')['Timestamp'].agg(lambda x: x.max() - x.min())

In [10]:
df_session_info = pd.read_csv('df_session_info2.csv', names=['temp', 'SessionID', 'Time', 'InBuys'])

# drop needless column, row, and convert str to timedelta for calculations
df_session_info = df_session_info.drop('temp', 1)
df_session_info = df_session_info.drop(df_session_info.index[0])
df_session_info.Time = pd.to_timedelta(df_session_info.Time)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
df_session_info.head()

,SessionID,Time,InBuys
1,1,00:05:51.029000,0
2,2,00:05:59.275000,0
3,3,00:12:25.378000,0
4,4,00:17:14.468000,0
5,6,00:04:06.128000,0


In [12]:
len(df_session_info)

9249729

In [14]:
df_session_info['seconds'] = df_session_info.Time.astype('timedelta64[s]')

In [16]:
df_session_info.InBuys = df_session_info.InBuys.apply(pd.to_numeric)
df_session_info.groupby(['InBuys'])['seconds'].mean()

InBuys
0    352.894155
1    866.630544
Name: seconds, dtype: float64

In [17]:
# sessions that result in buy(s) last on average 867 seconds vs. 353 seconds for sessions without a buy

In [20]:
# average number of items looked at during buy sessions vs. non-buy sessions
# create df to store number of clicks per session and merge into session info df
df_sessionID_counts = df_clicks.SessionID.value_counts()
df_sessionID_counts = pd.DataFrame({'SessionID':df_sessionID_counts.index, 'count':df_sessionID_counts.values})

In [21]:
df_sessionID_counts.head()

,SessionID,count
0,2574047,200
1,5300552,200
2,5488783,200
3,3972184,200
4,189448,200


In [22]:
df_session_info.SessionID = df_session_info.SessionID.astype(np.int64)

In [23]:
df_session_info_merged = df_session_info.merge(df_sessionID_counts, left_on='SessionID', right_on='SessionID')

In [24]:
len(df_session_info_merged)

9249729

In [25]:
df_session_info_merged.head()

,SessionID,Time,InBuys,seconds,count
0,1,00:05:51.029000,0,351.0,4
1,2,00:05:59.275000,0,359.0,6
2,3,00:12:25.378000,0,745.0,3
3,4,00:17:14.468000,0,1034.0,2
4,6,00:04:06.128000,0,246.0,2


In [26]:
df_session_info_merged.groupby(['InBuys'])['count'].mean()

InBuys
0    3.397957
1    6.485605
Name: count, dtype: float64

In [27]:
# above shows 3.4 items are clicked on in an average non-buy session, and 6.5 items are clicked on in an average buy session

In [38]:
# tables showing buy percentage by hour of day, day of week

In [30]:
df_clicks['Timestamp'] = pd.to_datetime(df_clicks['Timestamp'])
df_buys['Timestamp'] = pd.to_datetime(df_buys['Timestamp'])

df_clicks['hour'] = df_clicks.Timestamp.dt.hour
df_clicks['dayofweek'] = df_clicks.Timestamp.dt.dayofweek
df_buys['hour'] = df_buys.Timestamp.dt.hour
df_buys['dayofweek'] = df_buys.Timestamp.dt.dayofweek

In [31]:
df_hourb = df_buys.hour.value_counts()
df_hourc = df_clicks.hour.value_counts()
df_dowb = df_buys.dayofweek.value_counts()
df_dowc = df_clicks.dayofweek.value_counts()

df_hourb = df_hourb.to_frame()
df_hourc = df_hourc.to_frame()
df_dowb = df_dowb.to_frame()
df_dowc = df_dowc.to_frame()

df_hourb = df_hourb.reset_index()
df_hourc = df_hourc.reset_index()
df_dowb = df_dowb.reset_index()
df_dowc = df_dowc.reset_index()

df_hourb.columns = ['hour', 'buys']
df_hourc.columns = ['hour', 'clicks']
df_dowb.columns = ['dayofweek', 'buys']
df_dowc.columns = ['dayofweek', 'clicks']

df_hourb = df_hourb.reset_index(drop=True)
df_hourc = df_hourc.reset_index(drop=True)
df_dowb = df_dowb.reset_index(drop=True)
df_dowc = df_dowc.reset_index(drop=True)

df_hour_merged = df_hourc.merge(df_hourb, left_on='hour', right_on='hour')
df_dow_merged = df_dowc.merge(df_dowb, left_on='dayofweek', right_on='dayofweek')

In [34]:
df_hour_merged['buy_percent'] = df_hour_merged.buys / df_hour_merged.clicks
df_dow_merged['buy_percent'] = df_dow_merged.buys / df_dow_merged.clicks

In [35]:
df_hour_merged = df_hour_merged.sort_values(by='hour')
df_dow_merged = df_dow_merged.sort_values(by='dayofweek')

df_hour_merged = df_hour_merged.reset_index(drop=True)
df_dow_merged = df_dow_merged.reset_index(drop=True)

In [36]:
df_hour_merged

,hour,clicks,buys,buy_percent
0,0,155359,2837,0.018261
1,1,105322,1876,0.017812
2,2,102394,1757,0.017159
3,3,149191,2577,0.017273
4,4,391717,7533,0.019231
5,5,850077,19856,0.023358
6,6,1339638,39704,0.029638
7,7,1711323,60322,0.035249
8,8,1938499,73437,0.037883
9,9,2057423,77279,0.037561


In [37]:
df_dow_merged

,dayofweek,clicks,buys,buy_percent
0,0,6578482,215594,0.032773
1,1,2233498,40925,0.018323
2,2,4795554,147451,0.030747
3,3,4512049,146158,0.032393
4,4,3615548,126955,0.035114
5,5,3908733,171441,0.043861
6,6,7360080,302229,0.041063
